In [4]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# --- CONFIGURAÇÃO ---
PATH_MARKETAUX_CLEAN = "../data/news/processed/marketaux"
PATH_INVESTING_CLEAN = "../data/news/processed/investing"
OUTPUT_REPORT = "../data/news/reports"

# Cria pasta para salvar os gráficos
Path(OUTPUT_REPORT).mkdir(parents=True, exist_ok=True)

def carregar_dados(path, nome_fonte):
    print(f"📦 Carregando dados de: {nome_fonte}...")
    arquivos = glob.glob(os.path.join(path, "*.parquet"))
    
    if not arquivos:
        print(f"❌ Nenhum arquivo encontrado em {path}")
        return pd.DataFrame()
    
    dfs = []
    for f in arquivos:
        try:
            df = pd.read_parquet(f)
            dfs.append(df)
        except:
            pass
            
    if not dfs: return pd.DataFrame()
    
    full_df = pd.concat(dfs, ignore_index=True)
    full_df['source'] = nome_fonte
    return full_df

def gerar_relatorio(df, nome_fonte):
    if df.empty: return

    print(f"\n{'='*40}")
    print(f"📊 RELATÓRIO: {nome_fonte.upper()}")
    print(f"{'='*40}")
    
    # 1. Estatísticas Básicas
    total_news = len(df)
    tickers_unicos = df['ticker'].nunique()
    data_min = df['date'].min()
    data_max = df['date'].max()
    
    print(f"🔹 Total de Notícias: {total_news}")
    print(f"🔹 Tickers Cobertos: {tickers_unicos}")
    print(f"🔹 Período: {data_min} até {data_max}")
    
    # 2. Análise de Sentimento
    print("\n📈 Estatísticas de Sentimento:")
    print(df['sentiment'].describe().round(4))
    
    # Conta quantos são neutros exatos (0.0)
    neutros = (df['sentiment'] == 0.0).sum()
    pct_neutros = (neutros / total_news) * 100
    print(f"⚠️ Sentimentos Neutros (0.0): {neutros} ({pct_neutros:.2f}%)")
    
    # 3. Top Tickers por Volume
    print("\n🏆 Top 5 Tickers com mais notícias:")
    print(df['ticker'].value_counts().head(5))

    # --- GRÁFICOS ---
    plt.figure(figsize=(12, 5))
    
    # Gráfico 1: Distribuição de Sentimento
    plt.subplot(1, 2, 1)
    sns.histplot(df['sentiment'], bins=30, kde=True, color='skyblue')
    plt.title(f'Distribuição de Sentimento - {nome_fonte}')
    plt.xlabel('Score (-1 a 1)')
    plt.ylabel('Frequência')
    
    # Gráfico 2: Volume por Mês
    plt.subplot(1, 2, 2)
    df_temp = df.set_index('date')
    vol_mensal = df_temp.resample('M').size()
    vol_mensal.plot(kind='line', color='green')
    plt.title(f'Volume de Notícias por Mês - {nome_fonte}')
    plt.ylabel('Qtd Notícias')
    plt.grid(True, alpha=0.3)
    
    # Salvar
    filename = f"{OUTPUT_REPORT}/analise_{nome_fonte.lower()}.png"
    plt.tight_layout()
    plt.savefig(filename)
    print(f"\n💾 Gráfico salvo em: {filename}")
    plt.close()

def main():
    # 1. Carrega
    df_mkt = carregar_dados(PATH_MARKETAUX_CLEAN, "MarketAux")
    df_inv = carregar_dados(PATH_INVESTING_CLEAN, "Investing")
    
    # 2. Analisa Individualmente
    gerar_relatorio(df_mkt, "MarketAux")
    gerar_relatorio(df_inv, "Investing")
    
    # 3. Comparativo Rápido
    if not df_mkt.empty and not df_inv.empty:
        print(f"\n{'='*40}")
        print("⚔️ COMPARATIVO FINAL")
        print(f"{'='*40}")
        
        media_mkt = df_mkt['sentiment'].mean()
        media_inv = df_inv['sentiment'].mean()
        
        print(f"Média Sentimento MarketAux: {media_mkt:.4f}")
        print(f"Média Sentimento Investing: {media_inv:.4f}")
        
        if abs(media_mkt - media_inv) > 0.2:
            print("⚠️ AVISO: As fontes têm vieses muito diferentes!")
        else:
            print("✅ As fontes parecem alinhadas em termos de polaridade média.")

if __name__ == "__main__":
    main()

📦 Carregando dados de: MarketAux...
📦 Carregando dados de: Investing...

📊 RELATÓRIO: MARKETAUX
🔹 Total de Notícias: 204
🔹 Tickers Cobertos: 5
🔹 Período: 2022-01-03 18:04:38 até 2025-10-06 10:20:00

📈 Estatísticas de Sentimento:
count    204.0000
mean      -0.1189
std        0.1763
min       -0.5267
25%       -0.2960
50%        0.0000
75%        0.0000
max        0.6124
Name: sentiment, dtype: float64
⚠️ Sentimentos Neutros (0.0): 117 (57.35%)

🏆 Top 5 Tickers com mais notícias:
ticker
PETR4.SA    67
BBAS3.SA    53
VALE3.SA    50
WEGE3.SA    23
ITUB4.SA    11
Name: count, dtype: int64


C:\Users\kaike\AppData\Local\Temp\ipykernel_17804\459992911.py:81: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vol_mensal = df_temp.resample('M').size()



💾 Gráfico salvo em: ../data/news/reports/analise_marketaux.png

📊 RELATÓRIO: INVESTING
🔹 Total de Notícias: 23
🔹 Tickers Cobertos: 23
🔹 Período: 2021-10-25 00:00:00 até 2025-12-20 00:00:00

📈 Estatísticas de Sentimento:
count    23.0000
mean      0.3167
std       0.0000
min       0.3167
25%       0.3167
50%       0.3167
75%       0.3167
max       0.3167
Name: sentiment, dtype: float64
⚠️ Sentimentos Neutros (0.0): 0 (0.00%)

🏆 Top 5 Tickers com mais notícias:
ticker
ABEV3    1
AZZA3    1
B3SA3    1
CMIG4    1
COGN3    1
Name: count, dtype: int64


C:\Users\kaike\AppData\Local\Temp\ipykernel_17804\459992911.py:81: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vol_mensal = df_temp.resample('M').size()



💾 Gráfico salvo em: ../data/news/reports/analise_investing.png

⚔️ COMPARATIVO FINAL
Média Sentimento MarketAux: -0.1189
Média Sentimento Investing: 0.3167
⚠️ AVISO: As fontes têm vieses muito diferentes!


In [5]:
import pandas as pd
import glob
import os

PATH_INV = "../data/news/processed/investing"

def auditar_titulos():
    files = glob.glob(os.path.join(PATH_INV, "*.parquet"))
    if not files: return
    
    print(f"📂 Lendo {len(files)} arquivos do Investing...")
    
    # Lê todos e junta
    dfs = [pd.read_parquet(f) for f in files]
    df = pd.concat(dfs)
    
    print("\n🕵️ LISTA DE TÍTULOS CAPTURADOS:")
    # Mostra os títulos únicos e quantas vezes aparecem
    print(df['title'].value_counts())
    
    print("\n--- EXEMPLO DE UM REGISTRO ---")
    print(df.iloc[0])

if __name__ == "__main__":
    auditar_titulos()

📂 Lendo 23 arquivos do Investing...

🕵️ LISTA DE TÍTULOS CAPTURADOS:
title
Facebook","icon":"facebookIcon","href":"https://www.facebook.com/InvestingcomBrasil","target":"_blank"},{"title":"Twitter","icon":"twitterIcon","href":"https://twitter.com/InvestingBrasil","target":"_blank"}],"links":[{"children":[{"title":"Blog","href":"https://br.investing.com/blog/"},{"title":"Aplicativo para celular","href":"https://br.investing.com/mobile/"},{"title":"Portfólio","href":"https://br.investing.com/portfolio/"},{"title":"Widgets","href":"/webmaster-tools/"}]},{"children":[{"title":"Sobre Nós","href":"/about-us/"},{"title":"Anuncie","href":"https://br.investing.com/about-us/advertise"},{"title":"Ajuda e Suporte","href":"/about-us/contact-us"},{"title":"Expediente","href":"https://br.investing.com/about-us/editorial"}]}]},"quickLinksStore":{"links":[{"href":"/equities","name":"Ações"},{"href":"/stock-screener","name":"Filtro de Ações"},{"href":"/equities/brazil","name":"Brasil"},{"href":"/warrena